# Competition Twitter Sentiment Analysis

In this notebook, to predict the sentiment of a tweet, we use the algorithm : 

- Naive Bayesian.

#### Load libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV
from bs4 import BeautifulSoup
import re
import string
import nltk
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
from sklearn.metrics import confusion_matrix
#import ConfusionMatrix
from pandas_ml import ConfusionMatrix
from sklearn.metrics import accuracy_score

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


#### Load Train and Test Dataset

In [ ]:
train = pd.read_csv("input/train_E6oV3lV.csv", encoding='ISO-8859-1')
test = pd.read_csv("input/test_tweets_anuFYb8.csv", encoding='ISO-8859-1')

#### Function to clean data

In [ ]:
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

#### Clean Train and Test Dataset

In [ ]:
training = train.tweet
dftext = []
for t in training:
    dftext.append(tweet_cleaner(t))
    
testing = test.tweet
dftest = []
for t in testing:
    dftest.append(tweet_cleaner(t))

Transfrom Train label to list

In [ ]:
dfList = train['label'].tolist()

Merge Train tweet and label in dataframe

In [ ]:
dataset = sklearn.datasets.base.Bunch(data=dftext, target=dfList)
train = pd.DataFrame(dataset)
train.columns = ['tweet','label']

## Build model with pipeline

In [ ]:
pipeline = Pipeline([
    ('count_vectorizer',   CountVectorizer(stop_words = 'english', lowercase=True,binary= False,max_df= 0.5,max_features= None,min_df= 1)),
    ('tfidf_transformer',  TfidfTransformer(norm = 'l2', use_idf = True, sublinear_tf=True)),
    ('classifier',         MultinomialNB(alpha = 0.01, fit_prior=True))
])

## Test model with Train Dataset

In [ ]:
%%time
k_fold = KFold(n=len(train), n_folds=6)
scores = []
confusion = np.array([[0, 0], [0, 0]])
for train_indices, test_indices in k_fold:
    train_text = train.iloc[train_indices]['tweet'].values
    train_y = train.iloc[train_indices]['label'].values

    test_text = train.iloc[test_indices]['tweet'].values
    test_y = train.iloc[test_indices]['label'].values

    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, average= 'binary')
    scores.append(score)

print('Total tweets classified:', len(train))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

## Make predictions with Test Dataset

In [ ]:
predictions = pipeline.predict(dftest)
predictions = pd.DataFrame(predictions)

## Download Predictions

In [ ]:
frames = [test, predictions]
submission = pd.concat(frames, axis=1, join_axes=[test.index])
submission
submission = submission[['id',0]]
submission.describe()
submission.columns = ['id', 'label']
submission = submission[['id', 'label']]
submission.to_csv('sub11.csv')